In [13]:
import artm

In [14]:
artm.ARTM(num_topics=1)
print(artm.version())

0.10.1


In [54]:
from pathlib import *

In [55]:
!head -10 Train_1_cleaned.vw.txt

doc_ abstract paper describ provid custom servic allow user retriev identif number medic use spoken natur dialogu paper present evalu success retriev identif number approxim introduct current peopl deal custom servic center phone world wide web regular basi servic center support wide varieti task includ check balanc bank credit card account transfer money account anoth buy airlin ticket file incom tax return custom servic center use interact voic respons ivr front-end determin user need provid list option user choos rout appropri ivr gather essenti inform like user bank account number social secur number back-end support custom servic center use special comput exampl retriev account balanc databas case human oper howev ivr unwieldi use user need cover option provid forc user hit transfer human oper addit frequent user memor sequenc option desir inform therefor ani chang option great inconveni user moreov user alway hit speak live oper becaus prefer deal human instead machin final custo

In [77]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
import re

all_stopwords_gensim = STOPWORDS.union(set(['The', 'We']))

counter = 0
with open('Dataset/Train/TopMine_texts.vw.txt', 'r', errors="ignore") as fin:
    with open('Dataset/Train/TopMine_texts_cleaned.vw.txt', 'w') as fout:
        for line in fin.readlines():
            line = re.sub(r"\d+", "", line, flags=re.UNICODE)
            words = line.split()
            if len(words) < 5:
                continue
            else:
                words[0] = 'doc_' + str(counter)
                counter += 1
                clean = ' '.join([word for word in words if (len(word) > 2 and not word in all_stopwords_gensim)])
                fout.write(clean + '\n')

In [15]:
list_of_topics = []
for i in range(2, 151):
    list_of_topics.append('topic_' + str(i))

In [16]:
vw_filaname = 'Train_1_d.vw.txt'
bv = artm.BatchVectorizer(data_path=vw_filaname, data_format='vowpal_wabbit', batch_size=900, target_folder='batches')

In [25]:
tau_list = [0.1, 0.2, 0.05]
for phi_1 in tau_list:
    for phi_2 in tau_list:
        model = artm.ARTM(num_topics=150, dictionary=bv.dictionary)
        model.scores.add(artm.PerplexityScore(name='perplexity', dictionary=bv.dictionary))
        model.scores.add(artm.TopTokensScore(name='top-tokens', num_tokens=10))
        reg = artm.DecorrelatorPhiRegularizer(name='decor', tau = 0.01, topic_names=list_of_topics)
        reg_phi = artm.SmoothSparsePhiRegularizer(name='ssphi', tau=-0.1, topic_names=list_of_topics)
        reg_phi_add = artm.SmoothSparsePhiRegularizer(name='ssphi_add', tau=0.2, topic_names=['topic_0', 'topic_1'])
        reg_theta = artm.SmoothSparseThetaRegularizer(name='sstheta', tau=-phi_1, topic_names=list_of_topics)
        reg_theta_add = artm.SmoothSparseThetaRegularizer(name='sstheta_add', tau=phi_2, topic_names=['topic_0', 'topic_1'])
        model.regularizers.add(reg)
        model.regularizers.add(reg_phi)
        model.regularizers.add(reg_phi_add)
        model.regularizers.add(reg_theta)
        model.regularizers.add(reg_theta_add)
        
        name = '-1_2_-' + str(phi_1) + '_' + str(phi_2) + '.csv'
        print(name)
        for i in range(15):
            model.fit_offline(bv, num_collection_passes=1)
            print(f'Iter #{i}, perplexity: {model.score_tracker["perplexity"].last_value}')
        model.get_phi(topic_names=model.topic_names, class_ids=['@default_class'], model_name = model.model_pwt).to_csv(name)

-1_2_-0.1_0.1.csv
Iter #0, perplexity: 584311.4375
Iter #1, perplexity: 10441.974609375
Iter #2, perplexity: 7525.67578125
Iter #3, perplexity: 5772.8076171875
Iter #4, perplexity: 5097.7509765625
Iter #5, perplexity: 4777.77880859375
Iter #6, perplexity: 4600.9638671875
Iter #7, perplexity: 4491.705078125
Iter #8, perplexity: 4418.64013671875
Iter #9, perplexity: 4366.51513671875
Iter #10, perplexity: 4327.548828125
Iter #11, perplexity: 4297.6005859375
Iter #12, perplexity: 4274.07275390625
Iter #13, perplexity: 4255.1201171875
Iter #14, perplexity: 4239.5185546875
-1_2_-0.1_0.2.csv
Iter #0, perplexity: 584318.875
Iter #1, perplexity: 10438.6767578125
Iter #2, perplexity: 7526.46728515625
Iter #3, perplexity: 5775.455078125
Iter #4, perplexity: 5100.04638671875
Iter #5, perplexity: 4779.11669921875
Iter #6, perplexity: 4602.0048828125
Iter #7, perplexity: 4492.69677734375
Iter #8, perplexity: 4419.72265625
Iter #9, perplexity: 4367.71337890625
Iter #10, perplexity: 4328.89306640625
I

In [17]:
model = artm.ARTM(num_topics=150, dictionary=bv.dictionary)
model.scores.add(artm.PerplexityScore(name='perplexity', dictionary=bv.dictionary))
model.scores.add(artm.TopTokensScore(name='top-tokens', num_tokens=10))
model.scores.add(artm.SparsityPhiScore(name='sparse'))

reg = artm.DecorrelatorPhiRegularizer(name='decor', tau = 0.1, topic_names=list_of_topics)
reg_add = artm.DecorrelatorPhiRegularizer(name='decor_add', tau = 0.025, topic_names=['topic_0', 'topic_1'])
reg_phi = artm.SmoothSparsePhiRegularizer(name='ssphi', tau=-0.1, topic_names=list_of_topics)
reg_phi_add = artm.SmoothSparsePhiRegularizer(name='ssphi_add', tau=0.2, topic_names=['topic_0', 'topic_1'])
reg_theta = artm.SmoothSparseThetaRegularizer(name='sstheta', tau=-0.1, topic_names=list_of_topics)
reg_theta_add = artm.SmoothSparseThetaRegularizer(name='sstheta_add', tau=0.2, topic_names=['topic_0', 'topic_1'])
model.regularizers.add(reg)
model.regularizers.add(reg_add)
model.regularizers.add(reg_phi)
model.regularizers.add(reg_phi_add)
model.regularizers.add(reg_theta)
model.regularizers.add(reg_theta_add)

In [18]:
for i in range(15):
    model.fit_offline(bv, num_collection_passes=1)
    print(f'Iter #{i}, perplexity: {model.score_tracker["perplexity"].last_value}')
    print(f'sparse: {model.score_tracker["sparse"].last_value}')
#     model.regularizers['ssphi'].tau -= i * 0.001
#     model.regularizers['sstheta'].tau -= i * 0.001

Iter #0, perplexity: 584318.875
sparse: 0.9292151927947998
Iter #1, perplexity: 10438.6787109375
sparse: 0.9476853013038635
Iter #2, perplexity: 7526.56982421875
sparse: 0.9643049240112305
Iter #3, perplexity: 5775.4365234375
sparse: 0.9701709747314453
Iter #4, perplexity: 5100.06201171875
sparse: 0.9730623364448547
Iter #5, perplexity: 4779.15869140625
sparse: 0.9748249650001526
Iter #6, perplexity: 4602.07177734375
sparse: 0.9760282635688782
Iter #7, perplexity: 4492.78271484375
sparse: 0.9768936634063721
Iter #8, perplexity: 4419.81884765625
sparse: 0.9775543212890625
Iter #9, perplexity: 4367.81689453125
sparse: 0.9780692458152771
Iter #10, perplexity: 4329.0078125
sparse: 0.9784895777702332
Iter #11, perplexity: 4299.111328125
sparse: 0.9788404107093811
Iter #12, perplexity: 4275.595703125
sparse: 0.979134202003479
Iter #13, perplexity: 4256.69970703125
sparse: 0.9793873429298401
Iter #14, perplexity: 4241.14453125
sparse: 0.979605495929718


In [19]:
top_tokens = model.score_tracker['top-tokens'].last_tokens

for topic_name in model.topic_names:
    print(top_tokens[topic_name])

['lie', 'apl', 'ion', 'tire', 'aud', 'tha', 'thc', 'arc', 'rel', 'lit']
['ion', 'word', 'line', 'tha', 'NAME', 'VAT', 'Tel', 'Interests', 'son', 'thc']
['question', 'answer', 'type', 'correct', 'set', 'use', 'trec', 'passag', 'score', 'tion']
['featur', 'train', 'use', 'data', 'accuraci', 'perform', 'class', 'set', 'result', 'classif']
['object', 'spatial', 'group', 'refer', 'scene', 'domain', 'model', 'semant', 'ground', 'region']
['domain', 'corpus', 'data', 'WSJ', 'training', 'result', 'Table', 'evaluation', 'task', 'performance']
['opinion', 'data', 'set', 'image', 'metric', 'subjective', 'feature', 'human', 'source', 'result']
['candidate', 'model', 'distance', 'method', 'measure', 'probability', 'language', 'approach', 'based', 'performance']
['types', 'phrases', 'lexical', 'muc', 'text', 'concepts', 'developed', 'lexicon', 'group', 'produced']
['tile', 'ill', 'tbr', 'tim', 'ion', 'set', 'arc', 'ing', 'lhe', 'use']
['trees', 'grammars', 'notation', 'constraints', 'variables', 'no

In [20]:
model.get_phi(topic_names=model.topic_names, class_ids=['@default_class'], model_name = model.model_pwt).to_csv('Phi_150.csv')

In [4]:
import csv
import re
from pathlib import *

tau_list = [0.1, 0.2, 0.05]
for phi_1 in tau_list:
    for phi_2 in tau_list:
        name = '-1_2_-' + str(phi_1) + '_' + str(phi_2) + '.csv'
        print(name)
        terms = []
        j = 0
        with open(name) as f:
            reader = csv.reader(f)
            f = 0
            for row in reader:
                j += 1
                if j % 100000 == 0:
                    print(j)
                if f == 0:
                    f = 1
                    continue
                else:
                    count = 0
                    for i in range(1, 151):
                        if float(row[i]) > 0.03:
                            count += 1
                    if 0 < count < 5:
                        terms.append(re.sub(r'\'|\)', '', row[0].split(', ')[1]))
        
        precision = []
        recall = []
        iteration = 0
        terms = set(terms)
        for k in truth.keys():
            tp, fp, fn = 0, 0, 0
            truth_str = ' '.join(truth[k])
            truth_str = re.sub(r"\d+", "", truth_str, flags=re.UNICODE)
            sent = TextBlob(truth_str)
            gr_truth = set([w.lemmatize() for w in sent.words if len(w) > 2])
            fn = len(gr_truth)

            input_path = 'Dataset/TopMine_Stem_Texts/' + k + '.txt'
            with open(input_path, 'r') as fin:
                result = []
                words = set(fin.read().split())
                result = words & terms

            iteration += 1
            if iteration % 1000 == 0:
                print(iteration)

            match = result & gr_truth
            tp = len(match)
            fn = fn - tp
            fp = len(result) - tp
            if (tp + fp) >= 1:
                precision.append(tp / (tp + fp))
            else:
                precision.append(0)
            if (tp + fn) >= 1:
                recall.append(tp / (tp + fn))
            else:
                recall.append(0)
        print(result & gr_truth)
        prec = sum(precision)/len(precision)
        print(prec)
        rec = sum(recall)/len(recall)
        print(rec)
        f1 = 2 * prec * rec / (prec + rec)
        print(f1)

-1_2_-0.1_0.1.csv
100000
200000
300000
400000
500000
1000
2000
3000
4000
5000
6000
7000
8000
{'parsing', 'noun', 'phrase', 'clustering', 'method', 'document', 'text', 'algorithm', 'corpus', 'character', 'sentence', 'query', 'feature', 'case', 'concept', 'term', 'user'}
0.41307120601002917
0.2865197225446345
0.33834929101120675
-1_2_-0.1_0.2.csv
100000
200000
300000
400000
500000
1000
2000
3000
4000
5000
6000
7000
8000
{'parsing', 'noun', 'phrase', 'clustering', 'method', 'document', 'text', 'algorithm', 'corpus', 'character', 'sentence', 'query', 'feature', 'case', 'concept', 'term', 'user'}
0.41307120601002917
0.2865197225446345
0.33834929101120675
-1_2_-0.1_0.05.csv
100000
200000
300000
400000
500000
1000
2000
3000
4000
5000
6000
7000
8000
{'parsing', 'noun', 'phrase', 'clustering', 'method', 'document', 'text', 'algorithm', 'corpus', 'character', 'sentence', 'query', 'feature', 'case', 'concept', 'term', 'user'}
0.41307120601002917
0.2865197225446345
0.33834929101120675
-1_2_-0.2_0.

In [60]:
import csv
import re

terms = []
j = 0
with open('Phi_150.csv') as f:
    reader = csv.reader(f)
    f = 0
    for row in reader:
        j += 1
        if j % 100000 == 0:
            print(j)
        if f == 0:
            f = 1
            continue
        else:
            count = 0
            for i in range(1, 151):
                if float(row[i]) > 0.02:
                    count += 1
            if 0 < count < 4:
                terms.append(re.sub(r'\'|\)', '', row[0].split(', ')[1]))

100000
200000
300000
400000
500000


In [61]:
print(len(terms))

161


In [47]:
# with open('Terms_200_1_025_025.txt', 'r') as fin:
#     terms = fin.read().split()

In [24]:
import json
from textblob import TextBlob, Word

with open('Dataset/Train/Train_1.json', 'r') as fin:
    truth = json.load(fin)

In [62]:
from pathlib import *
precision = []
recall = []
iteration = 0
terms = set(terms)

for k in truth.keys():
    tp, fp, fn = 0, 0, 0
    truth_str = ' '.join(truth[k])
    truth_str = re.sub(r"\d+", "", truth_str, flags=re.UNICODE)
    sent = TextBlob(truth_str)
    gr_truth = set([w.lemmatize() for w in sent.words if len(w) > 2])
    fn = len(gr_truth)

    input_path = 'Dataset/TopMine_Stem_Texts/' + k + '.txt'
    with open(input_path, 'r') as fin:
        result = []
        words = set(fin.read().split())
        result = words & terms
    
    iteration += 1
    if iteration % 1000 == 0:
        print(iteration)
    
    match = result & gr_truth
    tp = len(match)
    fn = fn - tp
    fp = len(result) - tp
    if (tp + fp) >= 1:
        precision.append(tp / (tp + fp))
    else:
        precision.append(0)
    if (tp + fn) >= 1:
        recall.append(tp / (tp + fn))
    else:
        recall.append(0)

1000
2000
3000
4000
5000
6000
7000
8000


In [63]:
print(sorted(result))

['act', 'algorithm', 'answer', 'argument', 'article', 'belief', 'candidate', 'case', 'cation', 'character', 'class', 'cluster', 'clustering', 'concept', 'constraint', 'corpus', 'database', 'dictionary', 'distance', 'domain', 'entity', 'error', 'evalu', 'fragment', 'graph', 'input', 'instance', 'language', 'method', 'noun', 'pair', 'parsing', 'participant', 'person', 'phrase', 'query', 'question', 'role', 'score', 'segmentation', 'sens', 'set', 'similar', 'similarity', 'speaker', 'speech', 'state', 'stem', 'story', 'string', 'structure', 'summary', 'tagger', 'task', 'text', 'time', 'topic', 'train', 'training', 'transfer', 'translation', 'type', 'vector', 'web', 'wordnet']


In [64]:
print(sorted(gr_truth))

['algorithm', 'approach', 'bias', 'bigram', 'boosting', 'case', 'cd-rom', 'character', 'clustering', 'community', 'concept', 'convergence', 'corpus', 'disk', 'distribution', 'document', 'fact', 'feature', 'french', 'histogram', 'hypothesis', 'implementation', 'indexing', 'knowledge', 'learning', 'likelihood', 'measure', 'measuring', 'method', 'mining', 'nist', 'noise', 'normalization', 'noun', 'optimization', 'paragraph', 'parsing', 'phrase', 'precision', 'predicate-argument', 'process', 'processing', 'query', 'ranking', 'regression', 'representation', 'sampling', 'search', 'searching', 'sentence', 'spelling', 'statistic', 'summarizer', 'synonym', 'technique', 'technology', 'term', 'text', 'thesaurus', 'token', 'topic', 'training', 'trec', 'user', 'validation', 'weighting', 'word', 'wordnet', 'zoning']


In [65]:
print(result & gr_truth)

{'wordnet', 'character', 'algorithm', 'text', 'phrase', 'corpus', 'case', 'query', 'method', 'topic', 'noun', 'parsing', 'training', 'clustering', 'concept'}


In [66]:
prec = sum(precision)/len(precision)
print(prec)

0.29351709682776966


In [67]:
rec = sum(recall)/len(recall)
print(rec)

0.2474647257047873


In [68]:
f1 = 2 * prec * rec / (prec + rec)
print(f1)

0.26853075216507205
